To begin our project, we import numpy for making scientific computations, pandas for loading and modifying datasets, and matplotlib for plotting graphs.


In [1]:
#importing packages
import pandas as pd #to read and manipulate data
import numpy as np #data manipulation and mathematical functions
from keras.layers import Dense, LSTM, Dropout #to create a LSTM model
from keras.models import Sequential #to create a LSTM model
import matplotlib.pyplot as plt #for visualisation 

Using TensorFlow backend.


After making the necessary imports, we load data on Tata Global Beverage’s past stock prices. From the data, we select the values of the first and second columns (“Open” and “High” respectively) as our training dataset. The “Open” column represents the opening price for shares that day and the “High” column represents the highest price shares reached that day.


In [0]:
url = 'https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv'
dataset_train = pd.read_csv(url)
training_set = dataset_train.iloc[:, 1:2].values


To get a look at the dataset we’re using, we can check the head, which shows us the first five rows of our dataset.

In [3]:
dataset_train.head(3)

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.2,233.5,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.1,233.8,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.5,235.0,234.25,2240909,5248.60


“Low” represents the lowest share price for the day, “Last” represents the price at which the last transaction for a share went through. “Close” represents the price shares ended at for the day.


**Data Normalization**

Normalization is changing the values of numeric columns in the dataset to a common scale, which helps the performance of our model. To scale the training dataset we use Scikit-Learn’s MinMaxScaler with numbers between zero and one.


In [0]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

**Incorporating Timesteps Into Data**

We should input our data in the form of a 3D array to the LSTM model. First, we create data in 60 timesteps before using numpy to convert it into an array. Finally, we convert the data into a 3D array with X_train samples, 60 timestamps, and one feature at each step.


In [5]:
X_train = []
y_train = []
type(y_train)

list

In [0]:
for i in range(60, 2035):
  X_train.append(training_set_scaled[i-60:i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
#lets see what the first feature vector looks like in terms of its length
X_train[0].shape

(60, 1)

 **Creating the LSTM**
 
Before we can develop the LSTM, we have to make a few imports from Keras: Sequential for initializing the neural network, LSTM to add the LSTM layer, Dropout for preventing overfitting with dropout layers, and Dense to add a densely connected neural network layer.


In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [0]:
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')

model_1 = model.fit(X_train,y_train,epochs=100,batch_size=32)


Epoch 1/100
1975/1975 [==============================] - 24s 12ms/step - loss: 0.0135
Epoch 2/100
1975/1975 [==============================] - 22s 11ms/step - loss: 0.0030
Epoch 3/100
1975/1975 [==============================] - 22s 11ms/step - loss: 0.0028
Epoch 4/100
1975/1975 [==============================] - 21s 11ms/step - loss: 0.0023
Epoch 5/100
1975/1975 [==============================] - 21s 11ms/step - loss: 0.0024
Epoch 6/100
1760/1975 [=========================>....] - ETA: 2s - loss: 0.0022

**Plotting the Results**

After all these steps, we can use matplotlib to visualize the result of our predicted stock price and the actual stock price.


In [0]:
plt.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TATA Stock Price')
plt.legend()
plt.show()
